In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import csv
import os
import sys
import pandas 
sys.path.append("..")
import minimise
# Enable or disable Tensor Float 32 Execution
tf.config.experimental.enable_tensor_float_32_execution(False)
import matplotlib.pyplot as plt
from scipy import integrate

params = {"axes.labelsize": 14,
          "axes.titlesize": 16,}
plt.rcParams["axes.linewidth"] = 1
plt.rcParams['mathtext.bf'] = 'STIXGeneral:italic:bold'
plt.rcParams['figure.dpi'] = 100
plt.rcParams.update(params)

def place(ax):
  ax.tick_params(direction="in", which="minor", length=3)
  ax.tick_params(direction="in", which="major", length=5, labelsize=13)
  ax.grid(which="major", ls="dashed", dashes=(1, 3), lw=0.8, zorder=0)
  #ax.legend(frameon=True, loc="best", fontsize=12,edgecolor="black")
  fig.tight_layout()

In [2]:

def get_data_binary(path_to_data):
  data = pandas.read_csv(path_to_data, skiprows=4, header=None, delim_whitespace=True).to_numpy()
  num = len(data[:,0])
  return data[:,1], data[:,2],  data[:,4], data[:,6], data[:,8]


class BulkCorrelationEvaluator:
    """
    This class provides utilities to calculate c2 and c3 of the bulk fluid.
    """
    def __init__(self, modelH, modelO, rhoH_bulk, rhoO_bulk, dx=0.03, inputbins=667):
        self.modelH = modelH
        self.modelO = modelO
        self.dx = dx
        self.rhoH_bulk = rhoH_bulk
        self.rhoO_bulk = rhoO_bulk
        self.inputBins = inputbins
        
        rhoHWindow = np.full(self.inputBins, self.rhoH_bulk)
        rhoOWindow = np.full(self.inputBins, self.rhoO_bulk)
        inputBinsHalf = self.inputBins // 2
        self.xWindow = self.dx * np.linspace(-inputBinsHalf, inputBinsHalf, self.inputBins)
        inputHWindow = rhoHWindow.reshape(1, self.inputBins, 1)
        inputOWindow = rhoOWindow.reshape(1, self.inputBins, 1)
        self.inputHWindow = tf.Variable(inputHWindow)
        self.inputOWindow = tf.Variable(inputOWindow)

    def c2(self):
        with tf.GradientTape(persistent=True, watch_accessed_variables=False) as tape:
            tape.watch([self.inputHWindow, self.inputOWindow])
            c1H_result = self.modelH([self.inputHWindow, self.inputOWindow])
            
        c2HH = tf.squeeze(tape.gradient(c1H_result, self.inputHWindow)).numpy() / self.dx
        c2HO = tf.squeeze(tape.gradient(c1H_result, self.inputOWindow)).numpy() / self.dx

        with tf.GradientTape(persistent=True, watch_accessed_variables=False) as tape:
            tape.watch([self.inputOWindow, self.inputHWindow])
            c1O_result = self.modelO([self.inputOWindow, self.inputHWindow])
            
        c2OO = tf.squeeze(tape.gradient(c1O_result, self.inputOWindow)).numpy() / self.dx
        c2OH = tf.squeeze(tape.gradient(c1O_result, self.inputHWindow)).numpy() / self.dx
        
        return c2HH, c2HO, c2OO, c2OH
    
    
def get_c2r(c2, z):
    deriv_c2 = np.gradient(c2, z)
    r = z
    return r, - deriv_c2 / (2 * np.pi * r)



def Hankel_transform(c2, r, k):
    return np.trapz(c2 * r * np.sin(k * r), r) * 4 * np.pi / k

def get_c2k(c2, r):
    k = np.linspace(0.001, 100, 1000)
    c2k = np.array([Hankel_transform(c2, r, k_) for k_ in k])
    return k, c2k

In [ ]:


rhoH_bulk = 0.015
rhoO_bulk = 0.015


modelH_path = "../../models/RPM_H_PC_softplus.keras"
modelO_path = "../../models/RPM_O_PC_softplus.keras"

modelH = keras.models.load_model(modelH_path)
modelO = keras.models.load_model(modelO_path)
bulkcorr = BulkCorrelationEvaluator(modelH, modelO, rhoH_bulk, rhoO_bulk)

c2HH, c2HO, c2OO, c2OH = bulkcorr.c2()




z = bulkcorr.xWindow

c2HH = c2HH[z>0]
c2OO = c2OO[z>0]
c2HO = c2HO[z>0]
c2OH = c2OH[z>0]
z = z[z>0]






r, c2HH_r = get_c2r(c2HH, z)
r, c2OO_r = get_c2r(c2OO, z)
r, c2HO_r = get_c2r(c2HO, z)
r, c2OH_r = get_c2r(c2OH, z)


k, c2HH_k = get_c2k(c2HH_r, r)
k, c2OO_k = get_c2k(c2OO_r, r)
k, c2HO_k = get_c2k(c2HO_r, r)
k, c2OH_k = get_c2k(c2OH_r, r)






h_k = c2HH_k / (1 - rhoH_bulk * c2HH_k)
Sk = 1 + rhoH_bulk * h_k


In [4]:



C_HH = np.sqrt(rhoH_bulk*rhoH_bulk) * c2HH_k
C_OO = np.sqrt(rhoO_bulk*rhoO_bulk) * c2OO_k
C_HO = np.sqrt(rhoH_bulk*rhoO_bulk) * c2HO_k
C_OH = np.sqrt(rhoO_bulk*rhoH_bulk) * c2OH_k

C_matrix = np.array([[C_HH,C_HO],
                     [C_OH,C_OO]])

ones = 0*c2HH_k + 1

I_matrix = np.array([[ones,ones*0],
                     [ones*0,ones]])



IminusC = I_matrix - C_matrix
det_IminusC = IminusC[0,0]*IminusC[1,1] - IminusC[0,1]*IminusC[1,0]
inv_IminusC = np.array([[IminusC[1,1],-IminusC[0,1]],
                        [-IminusC[1,0],IminusC[0,0]]]) / det_IminusC

H_matrix = inv_IminusC - I_matrix

hk_HH = H_matrix[0,0] / np.sqrt(rhoH_bulk*rhoH_bulk)
hk_OO = H_matrix[1,1] / np.sqrt(rhoO_bulk*rhoO_bulk)
hk_HO = H_matrix[0,1] / np.sqrt(rhoH_bulk*rhoO_bulk)
hk_OH = H_matrix[1,0] / np.sqrt(rhoO_bulk*rhoH_bulk)


sk_HH = rhoH_bulk * rhoH_bulk / (rhoH_bulk + rhoO_bulk) * (hk_HH + 1/rhoH_bulk)
sk_OO = rhoO_bulk * rhoO_bulk / (rhoH_bulk + rhoO_bulk) * (hk_OO + 1/rhoO_bulk)
sk_HO = rhoH_bulk * rhoO_bulk / (rhoH_bulk + rhoO_bulk) * (hk_HO)
sk_OH = rhoO_bulk * rhoH_bulk / (rhoH_bulk + rhoO_bulk) * (hk_OH)


In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(9.5,3))


#c3 = bulkcorr.c3()

ax[0].plot(z, c2HH, color='hotpink', lw=2, label='HH')
ax[0].plot(z, c2OO, color='lightskyblue', lw=2, label='OO')
ax[0].plot(z, 0.5* (c2HO + c2OH), color='gray', lw=2, label='HO', ls='--')

ax[1].plot(r, c2HH_r, color='hotpink', lw=2)
ax[1].plot(r, c2OO_r, color='lightskyblue', lw=2, label='OO')
ax[1].plot(r, 0.5* (c2HO_r + c2OH_r), color='gray', lw=2, label='HO', ls='--')

ax[2].plot(k, c2HH_k, color='hotpink', lw=2)
ax[2].plot(k, c2OO_k, color='lightskyblue', lw=2, label='OO')
ax[2].plot(k, 0.5* (c2HO_k + c2OH_k), color='gray', lw=2, label='HO', ls='--')

ax[3].plot(k, sk_HH, color='hotpink', lw=2)
ax[3].plot(k, sk_OO, color='lightskyblue', lw=2)
ax[3].plot(k, 0.5* (sk_HO + sk_OH), color='gray', lw=2)


#ax[3].plot(k, Sk, color='hotpink', lw=2)


ax[0].set_xlim(0, 10)
ax[1].set_xlim(0, 10)
ax[2].set_xlim(0, 20)
ax[3].set_xlim(0, 20)

ax[3].set_ylim(-0.4, 1.0)

ax[1].set_ylim(-60, 20)
place(ax[0])
place(ax[1])
place(ax[2])
place(ax[3])



ax[0].set_ylabel(r"$c^{(2)} (z)$")
ax[0].set_xlabel(r"$z$")
ax[1].set_ylabel(r"$c^{(2)} (r)$")
ax[1].set_xlabel(r"$r$")
ax[2].set_ylabel(r"$\hat{c}^{(2)}(k)$")
ax[2].set_xlabel(r"$k$")
ax[3].set_ylabel(r"$S(k)$")
ax[3].set_xlabel(r"$k$")

ax[0].legend(frameon=True, loc="best", fontsize=12,edgecolor="black")

plt.tight_layout()